In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [3]:
np.unique(cg.get_roots([95302094482834765, 95302094482834732]))

array([864691136428220947], dtype=uint64)

In [7]:
children = cg.get_children(864691136428220947)
while children.size:
    parents = children.copy()
    children = cg.get_children(parents, flatten=True)
parents

array([95302094482842120, 95302094482842102, 95302094482850759,
       95302094482841210, 95302094482844888, 95302094482854311,
       95302094482844873, 95302094482851656, 95302094482844868,
       95302094482834662, 95302094482843884, 95302094482844867,
       95302094482843907, 95302094482844866, 95302094482844861,
       95302094482857170, 95302094482857169, 95302094482847147,
       95302094482853391, 95302094482844851, 95302094482847092,
       95302094482834765, 95302094482853394, 95302094482842125,
       95302094482851627, 95302094482846126, 95302094482838088,
       95302094482842134, 95302094482851628, 95302094482835926,
       95302094482856113, 95302094482850739, 95302094482853424,
       95302094482850738, 95302094482838074, 95302094482841204,
       95302094482849475, 95302094482850723, 95302094482841199,
       95302094482834732, 95302094482860164, 95302094482860149,
       95302094482846168, 95302094482846050, 95302094482842988,
       95302094482843008, 95302094482848

In [11]:
l2id_agglomeration_d, edges = cg.get_subgraph([864691136428220947])
l2ids = np.array(list(l2id_agglomeration_d.keys()), dtype=np.uint64)
children = cg.get_children(l2ids, flatten=True)
children

array([95302094482842120, 95302094482842102, 95302094482850759,
       95302094482841210, 95302094482844888, 95302094482854311,
       95302094482844873, 95302094482851656, 95302094482844868,
       95302094482834662, 95302094482843884, 95302094482844867,
       95302094482843907, 95302094482844866, 95302094482844861,
       95302094482857170, 95302094482857169, 95302094482847147,
       95302094482853391, 95302094482844851, 95302094482847092,
       95302094482834765, 95302094482853394, 95302094482842125,
       95302094482851627, 95302094482846126, 95302094482838088,
       95302094482842134, 95302094482851628, 95302094482835926,
       95302094482856113, 95302094482850739, 95302094482853424,
       95302094482850738, 95302094482838074, 95302094482841204,
       95302094482849475, 95302094482850723, 95302094482841199,
       95302094482834732, 95302094482860164, 95302094482860149,
       95302094482846168, 95302094482846050, 95302094482842988,
       95302094482843008, 95302094482848

In [13]:
len(parents) == len(children)
len(children)

196

In [14]:
np.sum(np.in1d(parents, children))

196

In [ ]:
data = {
  "sources": [
    [
      "95302094482834765",
      930180.9375,
      1031548.5625,
      595360
    ]
  ],
  "sinks": [
    [
      "95302094482834732",
      930355.1875,
      1031461.4375,
      595360
    ]
  ]
}

In [ ]:
from collections import defaultdict
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta._ws_cv.resolution

        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")

        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)
data_dict

In [ ]:
from pychunkedgraph.graph.operation import MulticutOperation

op = MulticutOperation(
    cg,
    user_id="test",
    source_ids=data_dict["sources"]["id"],
    sink_ids=data_dict["sinks"]["id"],
    source_coords=data_dict["sources"]["coord"],
    sink_coords=data_dict["sinks"]["coord"],
    bbox_offset=(240,240,24)
)

op._apply(operation_id="",timestamp=None)